In [1]:
#|default_exp toolloop

# Tool loop

In [2]:
import os
# os.environ['ANTHROPIC_LOG'] = 'debug'

In [3]:
#| export
from claudette.core import *
from fastcore.utils import *
from fastcore.meta import delegates
from functools import wraps

from anthropic.types import TextBlock, Message, ToolUseBlock

In [4]:
from IPython.display import display, Markdown, clear_output
from pprint import pprint

In [5]:
model = models[1]
model

'claude-sonnet-4-20250514'

## Problem setup

Anthropic provides an [interesting example](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/customer_service_agent.ipynb) of using tools to mock up a hypothetical ordering system. We're going to take it a step further, and show how we can dramatically simplify the process, whilst completing more complex tasks.

We'll start by defining the same mock customer/order data as in Anthropic's example, plus create a entity relationship between customers and orders:

In [6]:
def _get_orders_customers():
    orders = {
        "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
        "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
        "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

    customers = {
        "C1": dict(name="John Doe", email="john@example.com", phone="123-456-7890",
                   orders=[orders['O1'], orders['O2']]),
        "C2": dict(name="Jane Smith", email="jane@example.com", phone="987-654-3210",
                   orders=[orders['O3']])
    }
    return orders, customers

In [7]:
orders, customers = _get_orders_customers()

We can now define the same functions from the original example -- but note that we don't need to manually create the large JSON schema, since Claudette handles all that for us automatically from the functions directly. We'll add some extra functionality to update order details when cancelling too.

In [8]:
def get_customer_info(
    customer_id:str # ID of the customer
): # Customer's name, email, phone number, and list of orders
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")

def get_order_details(
    order_id:str # ID of the order
): # Order's ID, product name, quantity, price, and order status
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")

def cancel_order(
    order_id:str # ID of the order to cancel
)->bool: # True if the cancellation is successful
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True

We're now ready to start our chat.

## Manual tool use

In [9]:
tools = [get_customer_info, get_order_details, cancel_order]

In [10]:
chat = Chat(model, tools=tools)

We'll start with the same request as Anthropic showed:

In [11]:
r = chat('Can you tell me the email address for customer C1?')
print(r.stop_reason)
r.content

- Retrieving customer C1
tool_use


[TextBlock(citations=None, text="I'll retrieve the customer information for customer C1 to get their email address.", type='text'),
 ToolUseBlock(id='toolu_01TPFnVGd5gFg9CKL7pLGKcL', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

Claude asks us to use a tool. Claudette handles that automatically by just calling it again:

In [12]:
r = chat()
contents(r)

'The email address for customer C1 (John Doe) is john@example.com.'

Let's consider a more complex case than in the original example -- what happens if a customer wants to cancel all of their orders?

In [13]:
chat = Chat(model, tools=tools)
r = chat('Please cancel all orders for customer C1 for me.')
print(r.stop_reason)
r.content

- Retrieving customer C1
tool_use


[TextBlock(citations=None, text="I'll help you cancel all orders for customer C1. First, let me retrieve the customer's information to see what orders they have.", type='text'),
 ToolUseBlock(id='toolu_01KjiaEQVfXSpNGk2eEuHtcf', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

## Tool loop

This is the start of a multi-stage tool use process. Doing it manually step by step is inconvenient, so let's write a function to handle this for us:

In [14]:
#| exports
_final_prompt = "You have no more tool uses. Please summarize your findings. If you did not complete your goal please tell the user what further work needs to be done so they can choose how best to proceed."

In [16]:
@patch
@delegates(Chat.__call__)
def toolloop(self:Chat,
             pr, # Prompt to pass to Claude
             max_steps=10, # Maximum number of tool requests to loop through
             cont_func:callable=noop, # Function that stops loop if returns False
             final_prompt=_final_prompt, # Prompt to add if last message is a tool call
             **kwargs):
    "Add prompt `pr` to dialog and get a response from Claude, automatically following up with `tool_use` messages"
    class _Loop:
        def __iter__(a):
            init_n = len(self.h)
            r = self(pr, **kwargs)
            yield r
            for i in range(max_steps-1):
                if self.c.stop_reason!='tool_use': break
                r = self(final_prompt if i==max_steps-2 else None, **kwargs)
                yield r
                if not cont_func(*self.h[-3:]): break
            a.value = self.h[init_n:]
    return _Loop()

`toolloop` returns an iterable of assistant messages:

In [17]:
chat = Chat(model, tools=tools)
pr = 'Can you tell me the email address for customer C1?'
r = chat.toolloop(pr)
for o in r: display(o)

- Retrieving customer C1


I'll retrieve the customer information for customer C1 to get their email address.

<details>

- id: `msg_01HgJ8Zf97ZBquyY3K1fECG3`
- content: `[{'citations': None, 'text': "I'll retrieve the customer information for customer C1 to get their email address.", 'type': 'text'}, {'id': 'toolu_018oMD7g31SyjxJtnrDNxYY2', 'input': {'customer_id': 'C1'}, 'name': 'get_customer_info', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 574, 'output_tokens': 76, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

The email address for customer C1 (John Doe) is john@example.com.

<details>

- id: `msg_01RAWAfdXrc71FrhAJckuhoD`
- content: `[{'citations': None, 'text': 'The email address for customer C1 (John Doe) is john@example.com.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 787, 'output_tokens': 23, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

The full set of tool loop messages is stored in the `value` attr:

In [18]:
pprint(r.value, width=120)

[{'content': 'Can you tell me the email address for customer C1?', 'role': 'user'},
 {'content': [{'citations': None,
               'text': "I'll retrieve the customer information for customer C1 to get their email address.",
               'type': 'text'},
              {'id': 'toolu_018oMD7g31SyjxJtnrDNxYY2',
               'input': {'customer_id': 'C1'},
               'name': 'get_customer_info',
               'type': 'tool_use'}],
  'role': 'assistant'},
 {'content': [{'content': "{'name': 'John Doe', 'email': 'john@example.com', 'phone': '123-456-7890', 'orders': "
                          "[{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Shipped'}, "
                          "{'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': "
                          "'Processing'}]}",
               'tool_use_id': 'toolu_018oMD7g31SyjxJtnrDNxYY2',
               'type': 'tool_result'}],
  'role': 'user'},
 {'content': [{'citations':

Let's see if it can handle the multi-stage process now:

In [19]:
orders, customers = _get_orders_customers()

In [20]:
chat = Chat(model, tools=tools)
r = chat.toolloop('Please cancel all orders for customer C1 for me.')
for o in r: display(o)

- Retrieving customer C1


I'll help you cancel all orders for customer C1. First, let me retrieve the customer's information to see what orders they have.

<details>

- id: `msg_01VP6NoMuPvcGGwmmzqyyMYN`
- content: `[{'citations': None, 'text': "I'll help you cancel all orders for customer C1. First, let me retrieve the customer's information to see what orders they have.", 'type': 'text'}, {'id': 'toolu_012ZMh3bbykw2PVFtyECEvAZ', 'input': {'customer_id': 'C1'}, 'name': 'get_customer_info', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 574, 'output_tokens': 87, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

- Cancelling order O1
- Cancelling order O2


I can see that customer C1 (John Doe) has 2 orders:
- Order O1: Widget A (Status: Shipped)
- Order O2: Gadget B (Status: Processing)

Now I'll proceed to cancel both orders:

<details>

- id: `msg_01PPTjqVPhD1VY12UTqJvY4Z`
- content: `[{'citations': None, 'text': "I can see that customer C1 (John Doe) has 2 orders:\n- Order O1: Widget A (Status: Shipped)\n- Order O2: Gadget B (Status: Processing)\n\nNow I'll proceed to cancel both orders:", 'type': 'text'}, {'id': 'toolu_01BFhJ6A6YFswcP4jH5Xr5hE', 'input': {'order_id': 'O1'}, 'name': 'cancel_order', 'type': 'tool_use'}, {'id': 'toolu_0157kfvTrbDFkJcXtgEa5nCu', 'input': {'order_id': 'O2'}, 'name': 'cancel_order', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 798, 'output_tokens': 154, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

Perfect! I have successfully canceled all orders for customer C1 (John Doe):

✅ **Order O1** (Widget A) - Canceled successfully
✅ **Order O2** (Gadget B) - Canceled successfully

Both orders have been canceled. The customer will likely receive confirmation of these cancellations via email at john@example.com.

<details>

- id: `msg_01WUxHHvzHTZwwt6TR4q8t4j`
- content: `[{'citations': None, 'text': 'Perfect! I have successfully canceled all orders for customer C1 (John Doe):\n\n✅ **Order O1** (Widget A) - Canceled successfully\n✅ **Order O2** (Gadget B) - Canceled successfully\n\nBoth orders have been canceled. The customer will likely receive confirmation of these cancellations via email at john@example.com.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1017, 'output_tokens': 84, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

OK Claude thinks the orders were cancelled -- let's check one:

In [21]:
for o in chat.toolloop('What is the status of order O2?'): display(o)

- Retrieving order O2


Let me check the current status of order O2 for you.

<details>

- id: `msg_01U6VJY9whQ9Tk8yVPbQ9mmk`
- content: `[{'citations': None, 'text': 'Let me check the current status of order O2 for you.', 'type': 'text'}, {'id': 'toolu_01PvQe74e2JmGo1qKhJPPH98', 'input': {'order_id': 'O2'}, 'name': 'get_order_details', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1113, 'output_tokens': 73, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

The status of order O2 is **Cancelled**. This confirms that the cancellation we performed earlier was successful. The order for 1 Gadget B (priced at &#36;49.99) has been officially cancelled in the system.

<details>

- id: `msg_01XWuiYath9fYXb4sExiXS8P`
- content: `[{'citations': None, 'text': 'The status of order O2 is **Cancelled**. This confirms that the cancellation we performed earlier was successful. The order for 1 Gadget B (priced at $49.99) has been officially cancelled in the system.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1242, 'output_tokens': 55, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

If we run out of tool loops lets see what happens:

In [22]:
def mydiv(a:float, b:float):
    "Divide two numbers"
    return a / b

In [23]:
chat = Chat(model, tools=[mydiv])
r = chat.toolloop('Please calculate this sequence using your tools: 43/23454; 652/previous result; 6843/previous result; 321/previous result', max_steps=2)
for o in r: display(o)

I'll calculate this sequence step by step using the division tool.

<details>

- id: `msg_01AH41CspKKyBvnegC5RFp7Q`
- content: `[{'citations': None, 'text': "I'll calculate this sequence step by step using the division tool.", 'type': 'text'}, {'id': 'toolu_01Fa78R8zhaTkGVZx65t8vjY', 'input': {'a': 43, 'b': 23454}, 'name': 'mydiv', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 424, 'output_tokens': 84, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

I was able to complete the first two steps of your sequence:

**Completed calculations:**
1. 43 ÷ 23454 = 0.001833375969983798
2. 652 ÷ 0.001833375969983798 = 355628.0930232558

**Remaining work needed:**
I was unable to complete the full sequence due to tool usage limits. To finish the calculation, you would need:

3. 6843 ÷ 355628.0930232558 (divide 6843 by the result from step 2)
4. 321 ÷ [result from step 3] (divide 321 by the result from step 3)

You can either ask me to continue with these remaining calculations in a new conversation, or perform these final two divisions using a calculator with the intermediate result I provided: 355628.0930232558.

<details>

- id: `msg_01M1ZFznFinQp7PPurHaQQ5v`
- content: `[{'citations': None, 'text': 'I was able to complete the first two steps of your sequence:\n\n**Completed calculations:**\n1. 43 ÷ 23454 = 0.001833375969983798\n2. 652 ÷ 0.001833375969983798 = 355628.0930232558\n\n**Remaining work needed:**\nI was unable to complete the full sequence due to tool usage limits. To finish the calculation, you would need:\n\n3. 6843 ÷ 355628.0930232558 (divide 6843 by the result from step 2)\n4. 321 ÷ [result from step 3] (divide 321 by the result from step 3)\n\nYou can either ask me to continue with these remaining calculations in a new conversation, or perform these final two divisions using a calculator with the intermediate result I provided: 355628.0930232558.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 681, 'output_tokens': 212, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [24]:
chat = Chat(model, tools=mydiv)
r = chat.toolloop('Try dividing 1 by 0 and see what the error result is')
for o in r: display(o)

I'll try dividing 1 by 0 to see what error occurs:

<details>

- id: `msg_0124eQ1Wax74fJFRq2p4Y32E`
- content: `[{'citations': None, 'text': "I'll try dividing 1 by 0 to see what error occurs:", 'type': 'text'}, {'id': 'toolu_01SNv35VNUDsDLWhVbfYd5ff', 'input': {'a': 1, 'b': 0}, 'name': 'mydiv', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 409, 'output_tokens': 88, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

As expected, dividing 1 by 0 resulted in a `ZeroDivisionError: division by zero` error. This is the standard Python error that occurs when attempting to divide any number by zero, since division by zero is mathematically undefined.

The error traceback shows that the error occurred in the `mydiv` function at the line `return a / b` when `a=1` and `b=0`.

<details>

- id: `msg_01MLDokeUqvdSsoQ4JJJN3WG`
- content: `[{'citations': None, 'text': 'As expected, dividing 1 by 0 resulted in a `ZeroDivisionError: division by zero` error. This is the standard Python error that occurs when attempting to divide any number by zero, since division by zero is mathematically undefined.\n\nThe error traceback shows that the error occurred in the `mydiv` function at the line `return a / b` when `a=1` and `b=0`.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 701, 'output_tokens': 96, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

## Async tool loop

In [25]:
#| export
from claudette.asink import AsyncChat

In [26]:
#| exports
@patch
@delegates(AsyncChat.__call__)
def toolloop(
    self: AsyncChat,
    pr, # Prompt to pass to Claude
    max_steps=10, # Maximum number of tool requests to loop through
    cont_func: callable = noop, # Function that stops loop if returns False
    final_prompt = _final_prompt, # Prompt to add if last message is a tool call
    **kwargs
):
    "Add prompt `pr` to dialog and get a response from Claude, automatically following up with `tool_use` messages"
    class _Loop:
        async def __aiter__(a):
            init_n = len(self.h)
            r = await self(pr, **kwargs)
            yield r
            for i in range(max_steps-1):
                if self.c.stop_reason != 'tool_use': break
                r = await self(final_prompt if i==max_steps-2 else None, **kwargs)
                yield r
                if not cont_func(*self.h[-3:]): break
            a.value = self.h[init_n:]
    return _Loop()

In [27]:
orders, customers = _get_orders_customers()

In [28]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = AsyncChat(model, tools=tools)
r = chat.toolloop('Can you tell me the email address for customer C1?')
async for o in r: display(o)

- Retrieving customer C1


I'll retrieve the customer information for customer C1 to get their email address.

<details>

- id: `msg_015zT7mKFbxbcr2B3JT4VTEX`
- content: `[{'citations': None, 'text': "I'll retrieve the customer information for customer C1 to get their email address.", 'type': 'text'}, {'id': 'toolu_01YVi8Pgg7jafqq4FrdYdeyU', 'input': {'customer_id': 'C1'}, 'name': 'get_customer_info', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 574, 'output_tokens': 76, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

The email address for customer C1 (John Doe) is john@example.com.

<details>

- id: `msg_01FeEXsf8zKxRsLvxGqvB7TW`
- content: `[{'citations': None, 'text': 'The email address for customer C1 (John Doe) is john@example.com.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 787, 'output_tokens': 23, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [29]:
pprint(r.value)

[{'content': 'Can you tell me the email address for customer C1?',
  'role': 'user'},
 {'content': [{'citations': None,
               'text': "I'll retrieve the customer information for customer C1 "
                       'to get their email address.',
               'type': 'text'},
              {'id': 'toolu_01YVi8Pgg7jafqq4FrdYdeyU',
               'input': {'customer_id': 'C1'},
               'name': 'get_customer_info',
               'type': 'tool_use'}],
  'role': 'assistant'},
 {'content': [{'content': "{'name': 'John Doe', 'email': 'john@example.com', "
                          "'phone': '123-456-7890', 'orders': [{'id': 'O1', "
                          "'product': 'Widget A', 'quantity': 2, 'price': "
                          "19.99, 'status': 'Shipped'}, {'id': 'O2', "
                          "'product': 'Gadget B', 'quantity': 1, 'price': "
                          "49.99, 'status': 'Processing'}]}",
               'tool_use_id': 'toolu_01YVi8Pgg7jafqq4FrdYdeyU',


In [30]:
chat = AsyncChat(model, tools=tools)
async for o in chat.toolloop('Please cancel all orders for customer C1 for me.'): display(o)

- Retrieving customer C1


I'll help you cancel all orders for customer C1. First, let me retrieve the customer's information to see what orders they have.

<details>

- id: `msg_01Ukhoak7zdwxZeWPRTPKX4E`
- content: `[{'citations': None, 'text': "I'll help you cancel all orders for customer C1. First, let me retrieve the customer's information to see what orders they have.", 'type': 'text'}, {'id': 'toolu_015brcW6VQvQPifGZx8MbcLj', 'input': {'customer_id': 'C1'}, 'name': 'get_customer_info', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 574, 'output_tokens': 87, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

- Cancelling order O1
- Cancelling order O2


I can see that customer C1 (John Doe) has 2 orders:
- Order O1: Widget A (Status: Shipped)
- Order O2: Gadget B (Status: Processing)

Now I'll proceed to cancel both orders:

<details>

- id: `msg_01JGrnWb6epx8Hmv1NoxAYuy`
- content: `[{'citations': None, 'text': "I can see that customer C1 (John Doe) has 2 orders:\n- Order O1: Widget A (Status: Shipped)\n- Order O2: Gadget B (Status: Processing)\n\nNow I'll proceed to cancel both orders:", 'type': 'text'}, {'id': 'toolu_01EnfvnoNV7AmJAEkDDkiBfM', 'input': {'order_id': 'O1'}, 'name': 'cancel_order', 'type': 'tool_use'}, {'id': 'toolu_018pTUEhfy6RQazQkvrVCJuN', 'input': {'order_id': 'O2'}, 'name': 'cancel_order', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 798, 'output_tokens': 154, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

Perfect! I have successfully canceled all orders for customer C1 (John Doe):

✅ **Order O1** (Widget A) - Canceled successfully
✅ **Order O2** (Gadget B) - Canceled successfully

Both orders have been canceled. The customer will likely receive confirmation of these cancellations via email at john@example.com.

<details>

- id: `msg_01LGb8n7gAU1moD79LBnp5ux`
- content: `[{'citations': None, 'text': 'Perfect! I have successfully canceled all orders for customer C1 (John Doe):\n\n✅ **Order O1** (Widget A) - Canceled successfully\n✅ **Order O2** (Gadget B) - Canceled successfully\n\nBoth orders have been canceled. The customer will likely receive confirmation of these cancellations via email at john@example.com.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1017, 'output_tokens': 84, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

## Code interpreter

Here is an example of using `toolloop` to implement a simple code interpreter with additional tools.

In [39]:
from toolslm.shell import get_shell
from fastcore.meta import delegates
import traceback

In [40]:
@delegates()
class CodeChat(Chat):
    imps = 'os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses'
    def __init__(self, model: Optional[str] = None, ask:bool=True, **kwargs):
        super().__init__(model=model, **kwargs)
        self.ask = ask
        self.tools.append(self.run_cell)
        self.shell = get_shell()
        self.shell.run_cell('import '+self.imps)

We have one additional parameter to creating a `CodeChat` beyond what we pass to `Chat`, which is `ask` -- if that's `True`, we'll prompt the user before running code.

In [41]:
@patch
def run_cell(
    self:CodeChat,
    code:str,   # Code to execute in persistent IPython session
)->str:
    """Asks user for permission, and if provided, executes python `code` using persistent IPython session.
    Returns: Result of expression on last line (if exists); '#DECLINED#' if user declines request to execute"""
    confirm = f'Press Enter to execute, or enter "n" to skip?\n```\n{code}\n```\n'
    if self.ask and input(confirm): return '#DECLINED#'
    try: res = self.shell.run_cell(code)
    except Exception as e: return traceback.format_exc()
    return res.stdout if res.result is None else res.result

We just pass along requests to run code to the shell's implementation. Claude often prints results instead of just using the last expression, so we capture stdout in those cases.

In [65]:
sp = f'''You are a knowledgable assistant. Do not use tools unless needed.
Don't do complex calculations yourself -- use code for them.
The following modules are pre-imported for `run_cell` automatically:

{CodeChat.imps}

Never mention what tools you are using. Note that `run_cell` interpreter state is *persistent* across calls.

If a tool returns `#DECLINED#` report to the user that the attempt was declined and no further progress can be made.
In that case, do *not* attempt to run any further code -- stop execution *IMMEDIATELY* and tell the user it was declined.

When using a tool, *ALWAYS* before every use of every tool, tell the user what you will be doing and why.'''

In [66]:
def get_user()->str:
    "Get the username of the user running this session"
    print("Looking up username")
    return 'Jeremy'

In order to test out multi-stage tool use, we create a mock function that Claude can call to get the current username.

In [67]:
model = models[1]
chat = CodeChat(model, tools=[get_user], sp=sp, ask=True, temp=0.3)

Providing a callable to toolloop's `trace_func` lets us print out information during the loop:

`toolloop`'s `cont_func` callable let's us provide a function which, if it returns `False`, stops the loop:

In [68]:
def _cont_decline(call, resp, asst): return resp['content'][0]['content'] != '#DECLINED#'

Now we can try our code interpreter. We start by asking for a function to be created, which we'll use in the next prompt to test that the interpreter is persistent.

In [69]:
pr = '''Create a 1-line function `checksum` for a string `s`,
that multiplies together the ascii values of each character in `s` using `reduce`.'''
for o in chat.toolloop(pr, cont_func=_cont_decline): display(o)

Press Enter to execute, or enter "n" to skip?
```
checksum = lambda s: functools.reduce(operator.mul, (ord(c) for c in s), 1)

# Test the function
test_string = "hello"
result = checksum(test_string)
print(f"Checksum of '{test_string}': {result}")

# Let's verify by calculating manually
manual_calc = ord('h') * ord('e') * ord('l') * ord('l') * ord('o')
print(f"Manual calculation: {ord('h')} * {ord('e')} * {ord('l')} * {ord('l')} * {ord('o')} = {manual_calc}")
print(f"Results match: {result == manual_calc}")
```



I'll create a 1-line checksum function that multiplies the ASCII values of characters in a string using `reduce`.

<details>

- id: `msg_01NYULajnqbQD6QE5Up6LqNk`
- content: `[{'citations': None, 'text': "I'll create a 1-line checksum function that multiplies the ASCII values of characters in a string using `reduce`.", 'type': 'text'}, {'id': 'toolu_017igNofFN9FXpkiaKYTjBf4', 'input': {'code': 'checksum = lambda s: functools.reduce(operator.mul, (ord(c) for c in s), 1)\n\n# Test the function\ntest_string = "hello"\nresult = checksum(test_string)\nprint(f"Checksum of \'{test_string}\': {result}")\n\n# Let\'s verify by calculating manually\nmanual_calc = ord(\'h\') * ord(\'e\') * ord(\'l\') * ord(\'l\') * ord(\'o\')\nprint(f"Manual calculation: {ord(\'h\')} * {ord(\'e\')} * {ord(\'l\')} * {ord(\'l\')} * {ord(\'o\')} = {manual_calc}")\nprint(f"Results match: {result == manual_calc}")'}, 'name': 'run_cell', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 744, 'output_tokens': 250, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

Perfect! Here's the 1-line checksum function:

```python
checksum = lambda s: functools.reduce(operator.mul, (ord(c) for c in s), 1)
```

This function:
- Uses `functools.reduce` with `operator.mul` to multiply values together
- Converts each character to its ASCII value using `ord(c)`
- Uses a generator expression `(ord(c) for c in s)` to get ASCII values
- Starts with an initial value of `1` (the identity for multiplication)
- Returns the product of all ASCII values in the string

The function works correctly as demonstrated with the test string "hello", producing a checksum of 13,599,570,816.

<details>

- id: `msg_0136SzXuJJQWyYLHsHsKMHec`
- content: `[{'citations': None, 'text': 'Perfect! Here\'s the 1-line checksum function:\n\n```python\nchecksum = lambda s: functools.reduce(operator.mul, (ord(c) for c in s), 1)\n```\n\nThis function:\n- Uses `functools.reduce` with `operator.mul` to multiply values together\n- Converts each character to its ASCII value using `ord(c)`\n- Uses a generator expression `(ord(c) for c in s)` to get ASCII values\n- Starts with an initial value of `1` (the identity for multiplication)\n- Returns the product of all ASCII values in the string\n\nThe function works correctly as demonstrated with the test string "hello", producing a checksum of 13,599,570,816.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1052, 'output_tokens': 169, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

By asking for a calculation to be done on the username, we force it to use multiple steps:

In [70]:
pr = 'Use it to get the checksum of the username of this session.'
for o in chat.toolloop(pr): display(o)

Looking up username


I'll first get the username of this session, then use the checksum function to calculate its checksum.

<details>

- id: `msg_01VggyK78tiRhdvfML6t4Wnu`
- content: `[{'citations': None, 'text': "I'll first get the username of this session, then use the checksum function to calculate its checksum.", 'type': 'text'}, {'id': 'toolu_0191SgZiVKjPbJZePvHUVTjg', 'input': {}, 'name': 'get_user', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1238, 'output_tokens': 59, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

Press Enter to execute, or enter "n" to skip?
```
username = "Jeremy"
username_checksum = checksum(username)
print(f"Username: {username}")
print(f"Checksum: {username_checksum}")

# Let's also show the ASCII values for reference
ascii_values = [ord(c) for c in username]
print(f"ASCII values: {ascii_values}")
print(f"Manual verification: {' * '.join(map(str, ascii_values))} = {functools.reduce(operator.mul, ascii_values)}")
```



Now I'll calculate the checksum of the username "Jeremy":

<details>

- id: `msg_017BbU9iDY8eVPJoxUreJrio`
- content: `[{'citations': None, 'text': 'Now I\'ll calculate the checksum of the username "Jeremy":', 'type': 'text'}, {'id': 'toolu_01M9vkaS9zuqqzKicoFbDBaP', 'input': {'code': 'username = "Jeremy"\nusername_checksum = checksum(username)\nprint(f"Username: {username}")\nprint(f"Checksum: {username_checksum}")\n\n# Let\'s also show the ASCII values for reference\nascii_values = [ord(c) for c in username]\nprint(f"ASCII values: {ascii_values}")\nprint(f"Manual verification: {\' * \'.join(map(str, ascii_values))} = {functools.reduce(operator.mul, ascii_values)}")'}, 'name': 'run_cell', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1310, 'output_tokens': 185, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

The checksum of the username "Jeremy" is **1,134,987,783,204**.

This is calculated by multiplying the ASCII values: 74 × 101 × 114 × 101 × 109 × 121 = 1,134,987,783,204.

<details>

- id: `msg_01231J4En4o8QavmsiujeNBg`
- content: `[{'citations': None, 'text': 'The checksum of the username "Jeremy" is **1,134,987,783,204**.\n\nThis is calculated by multiplying the ASCII values: 74 × 101 × 114 × 101 × 109 × 121 = 1,134,987,783,204.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1578, 'output_tokens': 70, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

## Export -

In [71]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()